In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import cohere
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os
from tqdm import tqdm
import gradio as gr
from langchain import LLMChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage,HumanMessage
# os.environ['COHERE_API_KEY']=

In [ ]:
from langchain.chains import LLMChain
# from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
template = """Question: {question}

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])
llm=Cohere()
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

In [ ]:
for file in tqdm(os.listdir('./oldBios/')):
    azBookSummary=open(f'./oldBios/{file}','r').read()
    person=llm_chain.run(f"who is this biography ({azBookSummary})about? just give me the answer, no\
                  no explanations")
    print(person)
    wikiPersonDescription=llm_chain.run(f'give me wikipedia summary of {person}\
                                    in 100 words.')
    print(wikiPersonDescription)
    with open(f'./augBios/{file}','w') as f:
        f.write(wikiPersonDescription.replace('\n','')+'///'+azBookSummary)

In [ ]:
documents=[]
path='./oldBios/'
# path='./augBios/'
for file in os.listdir(path):
    loader = TextLoader(f'{path}{file}',encoding='unicode_escape')
    # loader.load()[0].metadata['category']='biography'
    # print(loader.load()[0].metadata)
    documents += loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=Cohere(), chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k':1}),return_source_documents=True)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings(model='embed-english-v3.0')
docsearch = Chroma.from_documents(texts, embeddings)
cohereLLM=Cohere(model='command')
memory=ConversationSummaryMemory(
    llm=cohereLLM, memory_key="chat_history", return_messages=True
)
# qa = ConversationalRetrievalChain.from_llm(llm=Cohere(model='command'), \
#     retriever=docsearch.as_retriever(),return_source_documents=True,memory=memory)
qa = ConversationalRetrievalChain.from_llm(llm=Cohere(model='command'), \
    retriever=docsearch.as_retriever(),memory=memory)
question_generator = LLMChain(llm=cohereLLM, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(cohereLLM, chain_type="refine")

chain = ConversationalRetrievalChain(
    retriever=docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True
)
# print(qa('hello'))
# print(chain({'question':'hello','chat_history':[]}))
def predict(message, history):
    # history_langchain_format = []
    # for human, ai in history:
    #     history_langchain_format.append(HumanMessage(content=human))
    #     history_langchain_format.append(AIMessage(content=ai))
    # history_langchain_format.append(HumanMessage(content=message))
    # gpt_response = llm(history_langchain_format)
    # return gpt_response.content
    
    # message=message+'? just the book title-Autho'
    #---------
    # for human, ai in history:
    #     history_langchain_format.append(HumanMessage(content=human))
    #     history_langchain_format.append(AIMessage(content=ai))
    # history_langchain_format.append(HumanMessage(content=message))
    # gpt_response = llm(history_langchain_format)
    # return gpt_response.content
    #------------
    message="you are a language model that gives book recommendation"+message+\
    'just give the book title and author'
        # chat_history=[]
        # result = chain({'question':message,'chat_history':chat_history})
    result=qa(message)
    # r1=docsearch.similarity_search_with_score(query=q,k=3)
    # print([(item[-2].metadata,item[-1]) for item in r1],\
    #       '\n\n',result['result'],f'|| {result["source_documents"][0].metadata}','\n*****\n')
    # if result['result'] not in ["I don't know","I don't know."]:
    #     return result['result']+f'\n---\nAmazon Kindle ebook description is:\n {result["source_documents"][0].page_content}'+\
    #     f'\nfrom this file: {result["source_documents"][0].metadata}'
    # else:
        # return result['result'] 
    # print(result)
    return result['answer']   
'''------'''
gr.ChatInterface(predict,
    chatbot=gr.Chatbot(height='auto'),
    textbox=gr.Textbox(placeholder="Recommend a book on someone who..."),
    title="Amazon But Better",
    description="Amazon started out with selling books. However, searching books on \
    Amazon is tedious and inaccurate if you don't know what you are exactly looking for. **Why not \
    make it faster and easier with LLMs:).**").launch()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = CohereEmbeddings(model='embed-english-v3.0')
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=Cohere(model='command'), chain_type="stuff", \
    retriever=docsearch.as_retriever(search_kwargs={'k':1}),return_source_documents=True)
# j=[HumanMessage(content='hello')]
# j.append(AIMessage(content='hi'))
# print(j)
# print(qa({'query': ''.join([HumanMessage(content='hello')].append(AIMessage(content='hi')))}))
history=[]
def predict(message, history):
    # gpt_response = qa({'query':''.join(history)+f'.\n given the previous conversation respond using the following prompt:{message}'})
    # # print(gpt_response)
    # history.append((f'HumanMessage:{message}',f'AIMessage: {gpt_response},'))
    # # history=history_langchain_format
    # return gpt_response['result']
    
    # message=message+'? just the book title-Author'
    # result = qa({"query": message})
    # # r1=docsearch.similarity_search_with_score(query=q,k=3)
    # # print([(item[-2].metadata,item[-1]) for item in r1],\
    # #       '\n\n',result['result'],f'|| {result["source_documents"][0].metadata}','\n*****\n')
    # if result['result'] not in ["I don't know","I don't know."]:
    #     return result['result']+f'\n---\nAmazon Kindle ebook description is:\n {result["source_documents"][0].page_content}'+\
    #     f'\nfrom this file: {result["source_documents"][0].metadata}'
    # else:
    #     return result['result']

gr.ChatInterface(predict,
    chatbot=gr.Chatbot(height='auto'),
    textbox=gr.Textbox(placeholder="Recommend a book on someone who..."),
    title="Amazon But Better",
    description="Amazon started out with selling books. However, searching books on \
    Amazon is tedious and inaccurate if you don't know what you are exactly looking for. **Why not \
    make it faster and easier with LLMs:).**").launch()

In [ ]:
qs=[
    'recommend a books on someone who was born in the state of mississpi?',
    'recommend a books on someone who has won mr.olympia?',
    'recommend a books on someone who started body building at 20?',
    'recommend a books on someone who played in magic mike?',
    'recommend a books on someone who was actor born in the uk?'
]
for q in qs:
    query = f"{q} Just give me the book title-Author"
    result = qa({"query": query})
    r1=docsearch.similarity_search_with_score(query=q,k=3)
    print([(item[-2].metadata,item[-1]) for item in r1],\
          '\n\n',result['result'],f'|| {result["source_documents"][0].metadata}','\n*****\n')